# CHAPTER 6 데이터 집계와 그룹 연산

## 목표
- groupby/agg/pivot_table로 의사결정용 KPI를 만든다.


In [ ]:
# ============================================
# [공통] 라이브러리 / 경로 / 출력 옵션 세팅
# 이 셀은 모든 챕터 노트북에서 동일하게 사용합니다.
# ============================================

# 1) 수치 계산(NumPy) / 표 데이터(Pandas) 불러오기
import numpy as np
import pandas as pd

# 2) 파일 경로를 운영체제와 무관하게 다루기 위한 Path
from pathlib import Path

# 3) 현재 노트북이 실행되는 폴더를 기준(BASE)으로 데이터 폴더(DATA) 지정
BASE = Path(".").resolve()          # 현재 작업 폴더(절대경로)
DATA = BASE / "data"                # data 폴더 경로

# 4) Pandas 출력 옵션(교육용): 너무 길게 출력되지 않도록 적당히 제한
pd.set_option("display.max_rows", 12)
pd.set_option("display.max_columns", 30)
pd.set_option("display.width", 140)

# 5) 확인 출력
print("BASE:", BASE)
print("DATA exists:", DATA.exists())


## 6.1 fact 테이블 만들기(조인 복습)

In [ ]:
# [이 셀은 무엇을 하는가?]
# - 분석에 필요한 한 장의 테이블(fact)을 재구성합니다.

customers = pd.read_csv(DATA / "customers.csv", parse_dates=["signup_date"])
orders    = pd.read_csv(DATA / "orders.csv", parse_dates=["order_date"])
items     = pd.read_csv(DATA / "order_items.csv")
products  = pd.read_csv(DATA / "products.csv")

orders_c = orders.merge(customers[["customer_id","region","age"]], on="customer_id", how="left")
items_p  = items.merge(products, on="product_id", how="left")
fact     = items_p.merge(orders_c, on="order_id", how="left")
fact["amount"] = fact["qty"] * fact["unit_price"]

display(fact.head())

## 6.2 groupby KPI

In [ ]:
# [이 셀은 무엇을 하는가?]
# - 지역/채널/카테고리 기준으로 매출을 집계해 비교합니다.

by_region = fact.groupby("region")["amount"].sum().sort_values(ascending=False)
by_channel = fact.groupby("channel")["amount"].sum().sort_values(ascending=False)
by_cat = fact.groupby("category")["amount"].sum().sort_values(ascending=False)

display(by_region)
display(by_channel)
display(by_cat)

## 6.3 agg로 여러 KPI 한 번에

In [ ]:
# [이 셀은 무엇을 하는가?]
# - region별로 총매출/주문수/구매자수 등을 한 번에 계산합니다.

kpi_region = fact.groupby("region").agg(
    total_amount=("amount","sum"),
    n_orders=("order_id","nunique"),
    n_customers=("customer_id","nunique"),
    avg_line_amount=("amount","mean")
).sort_values("total_amount", ascending=False)

display(kpi_region)

out_path = DATA / "kpi_region.csv"
kpi_region.to_csv(out_path)
print("saved:", out_path)

## 6.4 pivot_table 교차표

In [ ]:
# [이 셀은 무엇을 하는가?]
# - 지역(행)×카테고리(열) 매출 교차표를 만듭니다.

cross = fact.pivot_table(
    index="region",
    columns="category",
    values="amount",
    aggfunc="sum",
    fill_value=0
)

display(cross)

out_path = DATA / "region_category_sales.csv"
cross.to_csv(out_path)
print("saved:", out_path)